# **Практика: линейное программирование**

В языке Python есть множество библиотек, с помощью которых можно решить задачу линейного программирования. Вот основные, которые мы рассмотрим в данном юните:

+ **SciPy (scipy.optimize.linprog)**;
+ **CVXPY**; 
+ **PuLP**.

У каждой библиотеки есть свои особенности использования, и большинство задач можно решить с помощью любой из них. Давайте посмотрим все варианты, чтобы у вас всегда был выбор — решим по одной задаче для каждой библиотеки.

In [27]:
import numpy as np
from scipy import *
import cvxpy
import pulp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### **Пример № 1. SciPy (scipy.optimize.linprog)**

У нас есть 6 товаров с заданными ценами на них и заданной массой.

Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.

Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [28]:
# Создадим переменные на основе предложенных данных:

values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

Сформулируем задачу линейного программирования. Максимизируем произведение стоимости на количество, учитывая, что произведение веса на искомое количество товаров должно укладываться во вместимость сумки:

![](data/37.PNG)

Из предыдущего юнита мы уже знаем, что в векторно-матричной форме наша задача должна формулироваться в следующем виде:

![](data/38.PNG)

Получается, что в наших обозначениях мы имеем следующее:

![](data/39.PNG)

Здесь нам необходимо вспомнить линейную алгебру, так как очень важно, чтобы векторы были в нужных нам размерностях, иначе мы не сможем использовать матричное умножение. Вектор A размера 6 мы превращаем в матрицу размера (1,6) с помощью функции **expand_dims()**. Создаём все необходимые переменные:

In [29]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

# Передаём подготовленные переменные в оптимизатор SciPy:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.50000000003075
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24904539e-11])
  status: 0
 success: True
       x: array([6.18738537e-14, 1.05853307e-12, 1.21475944e-13, 7.50000000e+00,
       4.00246695e-13, 4.71394166e-13])

Получаем искомое значение функции — 52.5 (в выводе значение с минусом, но мы меняем знак, возвращаясь к задаче максимизации). x = (0,0,0,7.5,0,0). Таким образом, мы взяли только самую дорогую, четвёртую вещь. Она одна весит 2 кг, а если взять её 7.5 раз, то получится как раз 15 кг. Отлично, задача решена.
***

### **Пример № 2. CVXPY**

Снова решим задачу из примера № 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.

SciPy не умеет решать такие задачи, поэтому будем использовать новую библиотеку **CVXPY**.

Важно! С установкой этот библиотеки порой возникают проблемы. Если вы столкнулись с трудностями, посоветуйтесь с ментором или воспользуйтесь Google Colaboratory.

С помощью CVXPY создадим переменную-массив. Укажем его размерность, а также условие, что все числа в массиве должны быть целыми:

In [30]:
x = cvxpy.Variable(shape=n, integer = True)

# Далее зададим ограничения, используя матричное умножение:
constraint = (A @ x <= b)
total_value = c * x

# Переходим непосредственно к решению задачи:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

# Вызываем получившееся решение:
problem.solve()

c:\Users\Roman\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 12 times so far.

  warnings.warn(msg, UserWarning)


-inf

В результате получаем бесконечность. Это совершенно нереалистично.

В таком случае будем рассматривать только положительные значения x:

                                                x >= 0

В переформулированном виде задача будет решаться следующим образом:

In [31]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()
x.value

c:\Users\Roman\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 13 times so far.

  warnings.warn(msg, UserWarning)


array([0., 0., 0., 7., 0., 0.])

Здесь мы уже получаем 49, и берём только четвёртый товар в количестве семи штук. Можно увидеть, что результат, в целом, очень близок к первому, когда мы использовали библиотеку SciPy — различие лишь в добавлении целочисленности. Значит, у нас получилось решить задачу, когда мы добавили недостающее условие.

А что если мы можем брать не любое количество товаров, а только один или не брать их вовсе? Задаём x типа boolean.

x = 0  
или  
X = 1  

Программное решение такой задачи имеет следующий вид:

In [32]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
print(problem.solve())

x.value

-17.0


c:\Users\Roman\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\expressions\expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 14 times so far.

  warnings.warn(msg, UserWarning)


array([1., 0., 0., 1., 0., 1.])

Получим стоимость, равную 17, взяв первый, четвёртый и шестой товары.

Обратите внимание, что, используя SciPy, мы могли не указывать явно, что x только положительные, так как в линейном программировании считаются только неотрицательные x.

А вот CVXPY универсальна. Мы просто задали функцию, не указывая, что это линейное программирование. CVXPY «поняла», что это задача оптимизации, и использовала нужные алгоритмы. Поэтому здесь ограничение на положительные x мы указывали явно.
***

### **Пример № 3. PuLP**

В нашей каршеринговой компании две модели автомобилей: модель A и модель B. Автомобиль A даёт прибыль в размере 20 тысяч в месяц, а автомобиль B — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

* Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа A, и 5 дней — для каждого автомобиля типа B.
* Заводу требуется 3 дня, чтобы изготовить модель A, и 6 дней, чтобы изготовить модель B.
* Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль A, и 7 дней — автомобиль B.
* Каждый специалист может работать суммарно 30 дней.

![](data/40.PNG)

Заметьте, что здесь мы снова пишем обычные неравенства, а не условия в матричном виде. Дело в том, что для данной библиотеки так «удобнее», так как она принимает все условия в «первичном» виде.

In [33]:
problem = pulp.LpProblem('Производство машин', pulp.LpMaximize)
A = pulp.LpVariable('Автомобиль A', lowBound=0 , cat=pulp.LpInteger)
B = pulp.LpVariable('Автомобиль B', lowBound=0 , cat=pulp.LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", pulp.value(problem.objective))

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


c:\Users\Roman\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Выходит, что необходимо произвести 1 автомобиль типа A и 4 автомобиля типа B. Тогда суммарный чистый доход будет равен 200 тысячам.
***

### **ЗАДАЧИ**

Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:

![](https://lms.skillfactory.ru/assets/courseware/v1/e92f1aca3294f87b96e30eed2471af02/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md6_6_1_1.png.png)

Сформулируйте предложенную задачу как задачу линейного программирования и решите её любым способом (желательно программным).

В качестве ответа введите минимальную суммарную стоимость поставки. Ответ округлите до целого числа.

In [34]:
problem = pulp.LpProblem('Логистика', pulp.LpMinimize)
A = pulp.LpVariable('Склад 1 ТЦ1', lowBound=0 , cat=pulp.LpInteger)
B = pulp.LpVariable('Склад 1 ТЦ2', lowBound=0 , cat=pulp.LpInteger)
C = pulp.LpVariable('Склад 1 ТЦ3', lowBound=0 , cat=pulp.LpInteger)
D = pulp.LpVariable('Склад 2 ТЦ1', lowBound=0 , cat=pulp.LpInteger)
E = pulp.LpVariable('Склад 2 ТЦ2', lowBound=0 , cat=pulp.LpInteger)
F = pulp.LpVariable('Склад 2 ТЦ3', lowBound=0 , cat=pulp.LpInteger)
#Целевая функция
problem += 2*A + 5*B + 3*C + 7*D + 7*E + 6*F
#Ограничения
problem += A + B + C == 180 
problem += D + E + F == 220
problem += A + D <= 110
problem += B + E <= 150
problem += C + F <= 140
problem.solve()
print("Склад 1 ТЦ1: ", A.varValue)
print("Склад 1 ТЦ2: ", B.varValue)
print("Склад 1 ТЦ3: ", C.varValue)
print("Склад 2 ТЦ1: ", D.varValue)
print("Склад 2 ТЦ2: ", E.varValue)
print("Склад 2 ТЦ3: ", F.varValue)
print("Суммарная минимальная стоимость доставки: ", pulp.value(problem.objective))

Склад 1 ТЦ1:  110.0
Склад 1 ТЦ2:  0.0
Склад 1 ТЦ3:  70.0
Склад 2 ТЦ1:  0.0
Склад 2 ТЦ2:  150.0
Склад 2 ТЦ3:  70.0
Суммарная минимальная стоимость доставки:  1900.0


В прошлом юните мы обсуждали задачу о назначениях исполнителей задач - теперь пришло время решить её.

![](https://lms.skillfactory.ru/assets/courseware/v1/3c57d0c14951762fe62a494509d15a69/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md6_6_2.png)

Напомним суть: необходимо распределить пять задач между пятью исполнителями таким образом, чтобы суммарные затраты на работы были наименьшими.

В качестве ответа введите минимальную стоимость работ.

In [39]:
c = np.array([[1000,12,10,19,8],
          [12,1000,3,7,2],
          [10,3,1000,6,20],
          [19,7,6,1000,4],
          [8,2,20,4,1000]
          ])

x = cvxpy.Variable(shape=(5,5), boolean=True)
constraints = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5)
]
func = cvxpy.sum(cvxpy.multiply(x, c))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraints)
print(problem.solve())
x.value

32.0


array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])

Найдите кратчайший маршрут из точки A, который проходит через все другие точки и возвращается в A.

![](https://lms.skillfactory.ru/assets/courseware/v1/1dbdee74e890f6f4a8142e80d5799db3/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md6_6_3.png)

Сформулируйте эту задачу как задачу ЦЛП и решите её. В качестве ответа укажите длину кратчайшего пути.

In [42]:
# это то же самое что и выше
c = np.array([[1000,12,10,19,8],
          [12,1000,3,7,2],
          [10,3,1000,6,20],
          [19,7,6,1000,4],
          [8,2,20,4,1000]
          ])

x = cvxpy.Variable(shape=(5,5), boolean=True)
constraints = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5)
]
func = cvxpy.sum(cvxpy.multiply(x, c))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraints)
print(problem.solve())
x.value

32.0


array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])